# 预训练语言模型学习笔记

## 目录
1. [Word Structure and Subword Models](#1-word-structure-and-subword-models)
2. [Pretraining](#2-pretraining)
3. [Fine-tuning](#3-fine-tuning)
4. [BERT](#5-bert)
5. [T5](#6-t5)
6. [练习题](#8-练习题)

## 1. Word Structure and Subword Models

### 传统词汇表示的问题
- **词汇表过大**：自然语言中词汇数量庞大
- **OOV问题**：Out-of-Vocabulary，未见过的词无法处理
- **形态变化**：同一词根的不同形态被视为不同词
- **稀有词处理**：低频词缺乏足够训练数据

### Subword Models

#### Byte Pair Encoding (BPE)
- **核心思想**：从字符开始，逐步合并最频繁的字符对
- **优势**：平衡词汇表大小和语义完整性

#### WordPiece
- **Google提出**：用于BERT等模型
- **策略**：基于语言模型概率选择合并

#### SentencePiece
- **语言无关**：不依赖空格分词
- **统一处理**：将空格也作为特殊字符处理

In [1]:
import re
from collections import defaultdict, Counter

def get_word_tokens(vocab):
    """将词汇表转换为字符级tokens"""
    tokens = defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = ' '.join(list(word)) + ' </w>'
        tokens[word_tokens] += freq
    return tokens

def get_pairs(word_tokens):
    """获取所有相邻字符对"""
    pairs = defaultdict(int)
    for word, freq in word_tokens.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += freq
    return pairs

def merge_vocab(pair, word_tokens):
    """合并最频繁的字符对"""
    new_word_tokens = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in word_tokens:
        new_word = p.sub(''.join(pair), word)
        new_word_tokens[new_word] = word_tokens[word]
    return new_word_tokens

# 示例使用
vocab = {'low': 5, 'lower': 2, 'newest': 6, 'widest': 3}
word_tokens = get_word_tokens(vocab)
print("初始tokens:", word_tokens)

# 执行BPE
num_merges = 5
for i in range(num_merges):
    pairs = get_pairs(word_tokens)
    if not pairs:
        break
    best_pair = max(pairs, key=pairs.get)
    word_tokens = merge_vocab(best_pair, word_tokens)
    print(f"合并 {i+1}: {best_pair} -> {word_tokens}")

初始tokens: defaultdict(<class 'int'>, {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w e s t </w>': 6, 'w i d e s t </w>': 3})
合并 1: ('e', 's') -> {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}
合并 2: ('es', 't') -> {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}
合并 3: ('est', '</w>') -> {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}
合并 4: ('l', 'o') -> {'lo w </w>': 5, 'lo w e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}
合并 5: ('lo', 'w') -> {'low </w>': 5, 'low e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}


## 2. Pretraining

###  预训练的核心思想
- **无监督学习**：从大量无标注文本中学习语言表示
- **通用特征**：学习可迁移的语言特征
- **规模效应**：大数据+大模型=强能力

###  预训练任务类型

#### 语言建模 (Language Modeling)
- **目标**：预测下一个词
- **公式**：$P(w_t|w_1, w_2, ..., w_{t-1})$

#### 掩码语言建模 (Masked Language Modeling)
- **BERT使用**：随机掩码15%的词
- **双向上下文**：利用前后文信息

#### 下一句预测 (Next Sentence Prediction)
- **句子关系**：判断两个句子是否连续
- **应用**：问答、自然语言推理

## 3. Fine-tuning

###  微调策略

#### 全参数微调
- **更新所有参数**：适用于数据充足的情况
- **效果最好**：但计算成本高

#### 特征提取
- **冻结预训练参数**：只训练任务特定层
- **计算高效**：但效果可能受限

#### 渐进式微调
- **逐层解冻**：从顶层开始逐步解冻
- **平衡效果与效率**

### 3.2 学习率策略
- **较小学习率**：避免破坏预训练知识
- **差异化学习率**：不同层使用不同学习率
- **Warmup**：逐步增加学习率

## 4. BERT

###  BERT核心创新
- **双向编码器**：同时利用左右上下文
- **Transformer架构**：基于自注意力机制
- **预训练+微调**：两阶段训练范式

###  BERT架构
- **多层Transformer编码器**
- **位置编码**：处理序列位置信息
- **分段嵌入**：区分不同句子

### BERT预训练任务

#### Masked Language Model (MLM)
- **随机掩码15%的词**
- **80%替换为[MASK]，10%随机替换，10%保持不变**

#### Next Sentence Prediction (NSP)
- **判断两个句子是否连续**
- **50%正例，50%负例**

In [ ]:
# BERT模型使用示例
from transformers import BertTokenizer, BertForMaskedLM, BertForNextSentencePrediction
import torch.nn.functional as F

# 加载预训练模型
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
mlm_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
nsp_model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

# MLM示例
def predict_masked_word(text, model, tokenizer):
    """预测被掩码的词"""
    inputs = tokenizer(text, return_tensors='pt')
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits
    
    # 找到[MASK]位置
    mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
    
    # 获取预测结果
    mask_token_logits = predictions[0, mask_token_index, :]
    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
    
    print(f"原句: {text}")
    print("Top 5 预测:")
    for token in top_5_tokens:
        print(f"- {tokenizer.decode([token])}")

# NSP示例
def predict_next_sentence(sentence_a, sentence_b, model, tokenizer):
    """预测句子关系"""
    inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = F.softmax(outputs.logits, dim=-1)
    
    # 0: 不连续, 1: 连续
    is_next_prob = predictions[0][0].item()
    
    print(f"句子A: {sentence_a}")
    print(f"句子B: {sentence_b}")
    print(f"连续概率: {is_next_prob:.4f}")
    print(f"预测: {'连续' if is_next_prob > 0.5 else '不连续'}")

# 测试MLM
predict_masked_word("The capital of France is [MASK].", mlm_model, tokenizer)

print("\n" + "="*50 + "\n")

# 测试NSP
predict_next_sentence(
    "I went to the store.", 
    "I bought some milk.", 
    nsp_model, 
    tokenizer
)

## 5. T5 (Text-to-Text Transfer Transformer)

### T5核心思想
- **统一框架**：所有NLP任务都转换为文本到文本
- **编码器-解码器架构**：基于Transformer
- **任务前缀**：通过前缀指定任务类型

### T5架构特点
- **相对位置编码**：更好处理长序列
- **层归一化**：在每个子层之前
- **Gated Linear Unit (GLU)**：在前馈网络中

###  T5预训练
- **Span Corruption**：掩码连续的文本片段
- **去噪自编码器**：重构被破坏的文本

###  任务格式化
- **翻译**："translate English to German: Hello" → "Hallo"
- **摘要**："summarize: [long text]" → "[summary]"
- **问答**："question: What is the capital? context: [text]" → "Paris"

In [ ]:
# T5模型使用示例
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 加载T5模型
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

def t5_generate(task_prefix, input_text, model, tokenizer, max_length=50):
    """使用T5生成文本"""
    # 格式化输入
    input_text = f"{task_prefix}: {input_text}"
    
    # 编码
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    
    # 生成
    with torch.no_grad():
        outputs = model.generate(
            input_ids, 
            max_length=max_length,
            num_beams=4,
            early_stopping=True
        )
    
    # 解码
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print(f"任务: {task_prefix}")
    print(f"输入: {input_text}")
    print(f"输出: {generated_text}")
    print("-" * 50)

# 不同任务示例
tasks = [
    ("translate English to German", "Hello, how are you?"),
    ("summarize", "The quick brown fox jumps over the lazy dog. This is a common sentence used in typing practice."),
    ("cola sentence", "The book that I read yesterday was interesting."),  # 语法判断
]

for task_prefix, input_text in tasks:
    t5_generate(task_prefix, input_text, model, tokenizer)

# Span Corruption预训练任务示例
def create_span_corruption_data(text, tokenizer, noise_density=0.15):
    """创建span corruption训练数据"""
    tokens = tokenizer.encode(text, add_special_tokens=False)
    
    # 计算要掩码的token数量
    num_to_mask = int(len(tokens) * noise_density)
    
    # 随机选择起始位置
    start_idx = torch.randint(0, len(tokens) - num_to_mask + 1, (1,)).item()
    
    # 创建输入和目标
    input_tokens = tokens[:start_idx] + [tokenizer.additional_special_tokens_ids[0]] + tokens[start_idx + num_to_mask:]
    target_tokens = [tokenizer.additional_special_tokens_ids[0]] + tokens[start_idx:start_idx + num_to_mask] + [tokenizer.eos_token_id]
    
    input_text = tokenizer.decode(input_tokens)
    target_text = tokenizer.decode(target_tokens)
    
    print(f"原文: {text}")
    print(f"输入: {input_text}")
    print(f"目标: {target_text}")

# 示例
create_span_corruption_data("The quick brown fox jumps over the lazy dog", tokenizer)

## 6. 练习题

### 理论题

1. **解释BPE算法的工作原理，并说明其相比于词级别tokenization的优势。**

2. **BERT的MLM任务中，为什么要将15%的掩码词中的10%替换为随机词，10%保持不变？**

3. **比较BERT和T5的架构差异，说明各自的优缺点。**

4. **预训练语言模型存在哪些主要局限性？如何缓解这些问题？**

### 编程题

1. **实现一个简单的BPE算法**
2. **使用BERT进行文本分类任务的微调**
3. **实现T5风格的文本到文本任务**
4. **分析预训练模型中的偏见**

### 思考题

1. **如何设计更高效的预训练任务？**
2. **预训练模型的规模是否存在上限？**
3. **如何平衡模型性能和计算效率？**